# 1) Carga del dataset vehiculos a postgress

In [3]:
#librerias
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
df = pd.read_csv("../data/raw/used_cars_base.csv")


usuario = "etluser"
password = "etlpass"
host = "localhost"   
puerto = "5432"
base_datos = "dw"
engine = create_engine(f"postgresql://{usuario}:{password}@{host}:{puerto}/{base_datos}")


# Subir a PostgreSQL
df.to_sql("used_cars_base_raw", engine, if_exists="replace", index=False)
print("Dataset subido correctamente a PostgreSQL en la tabla 'used_cars_base_raw'")

Dataset subido correctamente a PostgreSQL en la tabla 'used_cars_base_raw'


# 2) Traer dataset desde postgress

In [ ]:
df_postgress = pd.read_sql("SELECT * FROM used_cars_base_raw", engine)

print(df_postgress.head())

   listing_id       brand    model  year  mileage_km  price_usd         city  \
0           1   Chevrolet     Onix  2016       90791   14551.54  Bucaramanga   
1           2     Hyundai      i30  2024      114658   21847.97     Medellín   
2           3  Volkswagen  T-Cross  2008       91468    5265.93     Medellín   
3           4       Mazda   Mazda2  2019       63498   19512.69         Cali   
4           5   Chevrolet  Tracker  2019       27864   18018.49         Cali   

             state      fuel transmission seller_type listing_date  \
0        Santander  gasoline       manual      dealer   2024-02-17   
1        Antioquia  gasoline    automatic     private   2024-04-22   
2        Antioquia  gasoline    automatic      dealer   2024-02-17   
3  Valle del Cauca    diesel    automatic      dealer   2024-07-12   
4  Valle del Cauca  electric    automatic      dealer   2024-07-13   

         source                                       url  
0  dataset_base      https://example.c

# 3) Limpieza del Dataset


**- Caracteristicas de las columnnas y vista de los primeros datos**

In [ ]:
pd.set_option("display.width", 200) 

print("Vista primeros 5 datos\n033[0m", df_postgress.head())
print("\n\033[1mCaracteristicas de las columnas:\033[0m")

df_postgress.info()

Vista primeros 5 datos
033    listing_id       brand    model  year  mileage_km  price_usd         city            state      fuel transmission seller_type listing_date        source                                       url
0           1   Chevrolet     Onix  2016       90791   14551.54  Bucaramanga        Santander  gasoline       manual      dealer   2024-02-17  dataset_base      https://example.com/Chevrolet/Onix/1
1           2     Hyundai      i30  2024      114658   21847.97     Medellín        Antioquia  gasoline    automatic     private   2024-04-22  dataset_base         https://example.com/Hyundai/i30/2
2           3  Volkswagen  T-Cross  2008       91468    5265.93     Medellín        Antioquia  gasoline    automatic      dealer   2024-02-17  dataset_base  https://example.com/Volkswagen/T-Cross/3
3           4       Mazda   Mazda2  2019       63498   19512.69         Cali  Valle del Cauca    diesel    automatic      dealer   2024-07-12  dataset_base        https://example.co

**Analisis** 

- No hay datos nulos
- La variable listing_date puede pasarse de tipo object a tipo DateTime
- La variable URL no aporta informacion realmente util


**- Cambio de tipo de variable listing_date de tipo object a DateTime**

In [ ]:
df_postgress["listing_date"] = pd.to_datetime(df_postgress["listing_date"], errors="coerce")

print("\n\033[1mCaracteristicas de las columnas:\033[0m")
df_postgress.info()

print("\n\033[1mVista primeros 5 datos\033[0m \n", df_postgress.head())


Caracteristicas de las columnas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   listing_id    300 non-null    int64         
 1   brand         300 non-null    object        
 2   model         300 non-null    object        
 3   year          300 non-null    int64         
 4   mileage_km    300 non-null    int64         
 5   price_usd     300 non-null    float64       
 6   city          300 non-null    object        
 7   state         300 non-null    object        
 8   fuel          300 non-null    object        
 9   transmission  300 non-null    object        
 10  seller_type   300 non-null    object        
 11  listing_date  300 non-null    datetime64[ns]
 12  source        300 non-null    object        
 13  url           300 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(9)
me

**- Eliminacion de la variable URL y Source**

In [10]:
df_postgress = df_postgress.drop(columns=["url"], errors="ignore")
df_postgress = df_postgress.drop(columns=["source"], errors="ignore")
print("\n\033[1mCaracteristicas de las columnas:\033[0m")
df_postgress.info()



Caracteristicas de las columnas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   listing_id    300 non-null    int64         
 1   brand         300 non-null    object        
 2   model         300 non-null    object        
 3   year          300 non-null    int64         
 4   mileage_km    300 non-null    int64         
 5   price_usd     300 non-null    float64       
 6   city          300 non-null    object        
 7   state         300 non-null    object        
 8   fuel          300 non-null    object        
 9   transmission  300 non-null    object        
 10  seller_type   300 non-null    object        
 11  listing_date  300 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(7)
memory usage: 28.2+ KB


**- Verificacion de duplicados**

In [11]:
print("Duplicados encontrados:", df_postgress.duplicated().sum())



Duplicados encontrados: 0


no se hallaron duplicados


**- Normalizacion de datos**

In [12]:
cols_texto = ["brand", "model", "city", "state", "fuel", "transmission", "seller_type"]
for col in cols_texto:
    df_postgress[col] = df_postgress[col].astype(str).str.strip().str.lower()


df_postgress.head()

,listing_id,brand,model,year,mileage_km,price_usd,city,state,fuel,transmission,seller_type,listing_date
0,1,chevrolet,onix,2016,90791,14551.54,bucaramanga,santander,gasoline,manual,dealer,2024-02-17
1,2,hyundai,i30,2024,114658,21847.97,medellín,antioquia,gasoline,automatic,private,2024-04-22
2,3,volkswagen,t-cross,2008,91468,5265.93,medellín,antioquia,gasoline,automatic,dealer,2024-02-17
3,4,mazda,mazda2,2019,63498,19512.69,cali,valle del cauca,diesel,automatic,dealer,2024-07-12
4,5,chevrolet,tracker,2019,27864,18018.49,cali,valle del cauca,electric,automatic,dealer,2024-07-13


# 4) guardar dataset limpio y subir a postgress

In [13]:
df_postgress.to_csv("../data/raw/used_cars_base_clean.csv", index=False)


df_postgress.to_sql("Used_cars_base_clean", engine, if_exists="replace", index=False)
print("Dataset subido correctamente a PostgreSQL en la tabla 'Used_cars_base_clean'")


Dataset subido correctamente a PostgreSQL en la tabla 'Used_cars_base_clean'


In [33]:
df2 = pd.read_csv("used_cars_base_clean.csv")

df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   listing_id    300 non-null    int64  
 1   brand         300 non-null    object 
 2   model         300 non-null    object 
 3   year          300 non-null    int64  
 4   mileage_km    300 non-null    int64  
 5   price_usd     300 non-null    float64
 6   city          300 non-null    object 
 7   state         300 non-null    object 
 8   fuel          300 non-null    object 
 9   transmission  300 non-null    object 
 10  seller_type   300 non-null    object 
 11  listing_date  300 non-null    object 
dtypes: float64(1), int64(3), object(8)
memory usage: 28.3+ KB
